### Data
[Amazon food reviews](https://storage.googleapis.com/kaggle-data-sets/18/2157/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1571493549&Signature=KDGN0DguhOFma%2BAD4p3tcUTMqceTUuKJIrJ8NdxuBNSHewMyhYl2Al3l5YcS79VrFadhLEuR%2BStitywsXf%2BvbKeaj9QONKCex9%2FoG4wSuYAUUCj%2B%2F15Z8YwrCVnLi%2F9YjqBPgUHC6lORqTZpXxeko8AtvC8F7ZWqlaWVSySaedSMMC3ZTae2qeGMPd5JDOfTy1J%2BNUxfCeDRsCvF7IilhOSsGgvJQ%2F4BnITZv%2FZ1PTu%2Bt%2FF836xVrj%2FZrtrNErTUR3htcsspgVBEeRrrl5qXeIbK2LCiRJUftd5tZjgYjWZKjyp94UxmcImGDcsz5qmWv0jwYDvJa2EUXmdN3HraYg%3D%3D&response-content-disposition=attachment%3B+filename%3Damazon-fine-food-reviews.zip)

In [15]:
import pandas as pd

reviews = pd.read_csv('amazon-fine-food-reviews/Reviews.csv')


In [16]:
print(reviews.shape)
print(reviews.head())
print(reviews.isnull().sum())

(568454, 10)
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality can

In [18]:
reviews=reviews.dropna()
#reviews=reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator', 'Score','Time'], 0)
reviews=reviews.reset_index(drop=True)

In [20]:
#print(reviews.head())
print(reviews.shape)
print(reviews.isnull().sum())
for i in range(5):
    print('Review #{}'.format(i+1))
    print(reviews.Summary[1])
    print(reviews.Text[i])
    print()

(568411, 10)
Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64
Review #1
Not as Advertised
I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

Review #2
Not as Advertised
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

Review #3
Not as Advertised
This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into t

In [21]:
import re
from nltk.corpus import stopwords
from pickle import dump, load
#import nltk

#nltk.download('stopwords')


contractions = {

"ain't": "am not",

"aren't": "are not",

"can't": "cannot",

"can't've": "cannot have",

"'cause": "because",

"could've": "could have",

"couldn't": "could not",

"couldn't've": "could not have",

"didn't": "did not",

"doesn't": "does not",

"don't": "do not",

"hadn't": "had not",

"hadn't've": "had not have",

"hasn't": "has not",

"haven't": "have not",

"he'd": "he would",

"he'd've": "he would have"
}
def clean_text(text, remove_stopwords=True, generate_corpus=True):
    # Convert words to lower case
    text = text.lower()
    if generate_corpus:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text)
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)
        if remove_stopwords:
            text = text.split()
            stops = set(stopwords.words("english"))
            text = [w for w in text if not w in stops]
            text = " ".join(text)
        return text

ModuleNotFoundError: No module named 'nltk'

In [ ]:
# Clean the summaries and texts

clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
    print("Summaries are complete.")


clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
    print("Texts are complete.")

In [ ]:
stories = list()
for i, text in enumerate(clean_texts):
    stories.append({'story': text, 'highlights': clean_summaries[i]})
# save to file
    dump(stories, open('review_dataset.pkl', 'wb'))

In [22]:
batch_size=64
epochs=110
latent_dim=256
num_samples=10000

In [ ]:
#Load data
stories=load('review_dataset.pkl','rb')
print('Loaded data length: {}, \nOf datatype: {}'.format(len(stories),type(stories)))

In [ ]:
#Vectorise the data

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for story in stories:
    input_text = story['story']
    input_texts.append(input_text)
    
    for highlight in story['highlights']:
        target_text = highlight    
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target_text + '\n'
        target_texts.append(target_text)    

for char in input_text:
    if char not in input_characters:
        input_characters.add(char)

for char in target_text:
    if char not in target_characters:
        target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

In [ ]:
#create a generic function to define an encoder-decoder RNN

def define_models(n_input, n_output, n_units):
    
    # define training encoder    
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,  initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return [model, encoder_model, decoder_model]

In [ ]:
#Training the model, using rmsprop optimizer and categorical_crossentropy loss function

model, encoder, decoder = define_models()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

# Save model
model.save('deeplearning_keras_summarization_model.h5')

In [ ]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    
    # encode
    state = infenc.predict(source)
    
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)

    # collect predictions
    output = list()

    for t in range(n_steps):

        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)

        # store prediction
        output.append(yhat[0,0,:])

        # update state
        state = [h, c]

        # update target sequence
        target_seq = yhat

    return array(output)